In [1]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import torch
from torchvision import transforms
from skimage import img_as_ubyte

# Function to calculate PSNR
def psnr(target, ref):
    mse = np.mean((target - ref) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

# Function to calculate SSIM
def calculate_ssim(target, ref):
    return ssim(target, ref, multichannel=True)

# Load model from torchhub
preprocess, postprocess, model = torch.hub.load('tnwei/waternet', 'waternet')
model.eval()



/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/tnwei/waternet/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://www.dropbox.com/s/j8ida1d86hy5tm4/waternet_exported_state_dict-daa0ee.pt?dl=1" to /root/.cache/torch/hub/checkpoints/waternet_exported_state_dict-daa0ee.pt


WaterNet(
  (cmg): ConfidenceMapGenerator(
    (conv1): Conv2d(12, 128, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (relu1): ReLU()
    (conv2): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu2): ReLU()
    (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu3): ReLU()
    (conv4): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (relu4): ReLU()
    (conv5): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (relu5): ReLU()
    (conv6): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu6): ReLU()
    (conv7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu7): ReLU()
    (conv8): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (sigmoid): Sigmoid()
  )
  (wb_refiner): Refiner(
    (conv1): Conv2d(6, 32, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), pad

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Input and output directories
input_folder = "/content/drive/MyDrive/aqua_data/non_invasive"
output_folder = "/content/drive/MyDrive/SRCNN/water/non_inv"

# Ensure output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the directory
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        file_path = os.path.join(input_folder, filename)
        im = cv2.imread(file_path)
        if im is None:
            continue  # Skip files that can't be opened

        # Preprocess the image
        rgb_im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        rgb_im = cv2.resize(rgb_im, (720, 480))

        # Inference
        rgb_ten, wb_ten, he_ten, gc_ten = preprocess(rgb_im)
        out_ten = model(rgb_ten, wb_ten, he_ten, gc_ten)
        out_im = postprocess(out_ten)[0]

        # Convert tensor to numpy array for evaluation
        rgb_im_np = np.array(rgb_im)
        out_im_np = np.array(out_im)

        # Calculate PSNR, MSE, and SSIM
        psnr_value = psnr(rgb_im_np, out_im_np)
        mse_value = np.mean((rgb_im_np - out_im_np) ** 2)
        ssim_value = calculate_ssim(rgb_im_np, out_im_np)

        # Print metrics
        print(f"PSNR: {psnr_value}, MSE: {mse_value}, SSIM: {ssim_value}")

        # Save the processed image
        output_file_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_file_path, cv2.cvtColor(out_im_np, cv2.COLOR_RGB2BGR))


<ipython-input-1-4595a9bfc546>:19: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return ssim(target, ref, multichannel=True)


PSNR: 27.92276957574923, MSE: 104.90674286265433, SSIM: 0.883194141153778
PSNR: 27.769973337857454, MSE: 108.66333429783951, SSIM: 0.6362336243728819
PSNR: 27.835408974528907, MSE: 107.03836419753087, SSIM: 0.8991405413928161
PSNR: 28.098066901336782, MSE: 100.75663869598766, SSIM: 0.9170284273406092
PSNR: 27.56532060020723, MSE: 113.90644579475308, SSIM: 0.7356433177551698
PSNR: 28.063945758629274, MSE: 101.55136959876543, SSIM: 0.9063733947855391
PSNR: 28.854677044026733, MSE: 84.64721161265432, SSIM: 0.9604736634459926
PSNR: 27.7887626685366, MSE: 108.19422839506173, SSIM: 0.8838838859349633
PSNR: 27.89470769530121, MSE: 105.58679108796296, SSIM: 0.9160704168591328
PSNR: 27.68414361373108, MSE: 110.83221161265432, SSIM: 0.7486128511777222
PSNR: 27.705539698832986, MSE: 110.2875250771605, SSIM: 0.8979730948053394
PSNR: 27.73249218658382, MSE: 109.60519579475309, SSIM: 0.8546470632386004
PSNR: 27.93443208345571, MSE: 104.62540509259259, SSIM: 0.8261171452995386
PSNR: 27.79131399703079